In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
CHP_ID = "1740"
SUBMODEL = "cond-detr-50" # "cond-detr-50"
MODEL_PATH = f"logs/{SUBMODEL}/checkpoint-{CHP_ID}"
# IMAGE_SHAPE = 1333 _Shape{IMAGE_SHAPE}
THR = 0.05
iou_threshold = 0.8
FILE_NAME = f"{SUBMODEL}_THR{THR*100:.3f}_IOU{iou_threshold:.3f}_ID{CHP_ID}_P"

In [3]:
from transformers import (
	AutoImageProcessor,
	AutoModelForObjectDetection,
	ConditionalDetrImageProcessor,
    ConditionalDetrForObjectDetection
)
from PIL import Image
import torch
from torchvision.ops import nms

import pandas as pd
import numpy as np

In [ ]:
from zindi_code.dataset import load_and_format
from zindi_code import CLS_MAPPER

image_folder = "zindi_data/images"

test = pd.read_csv("zindi_data/Test.csv")
test.sample(5)

In [5]:
model_pth = MODEL_PATH

image_processor: ConditionalDetrImageProcessor = AutoImageProcessor.from_pretrained(
    model_pth
)
model: ConditionalDetrForObjectDetection = AutoModelForObjectDetection.from_pretrained(
    model_pth
).to(
    "cuda"
)  # .train(False)

In [ ]:
image_processor

In [ ]:
model.config.id2label

In [8]:
@torch.no_grad()
def make_predictions(images: list[Image.Image]):
	inputs = image_processor(images=images, return_tensors="pt").to("cuda")
	outputs = model(**inputs)
	target_sizes = torch.tensor([image.size[::-1] for image in images])
	return image_processor.post_process_object_detection(
		outputs, threshold=THR, target_sizes=target_sizes
	)

def load_transform(path: str):
	return Image.open(os.path.join(image_folder, path)).convert("RGB")
	image = Image.open(os.path.join(image_folder, path))
	return np.array(image.convert("RGB"))[:, :, ::-1]

def load_images(image_pths: list[str]):
	return [
		load_transform(image_pth)
		for image_pth in image_pths
	]


def predicts(image_pths: list[str]):
	images = load_images(image_pths)
	results = make_predictions(images)
	predictions = []
	for image_pth, result in zip(image_pths, results):
		prediction = []
		if len(result["boxes"]):
			indices = nms(result["boxes"], result["scores"], iou_threshold)
			if not len(indices):
				continue
			for score, label, box in zip(
				result["scores"][indices],
				result["labels"][indices],
				result["boxes"][indices],
			):
				x1, y1, x2, y2 = (round(i, 2) for i in box.tolist())
				prediction.append(
					[
						image_pth,
						model.config.id2label[label.item()],
						round(score.item(), 3),
						x1,
						y1,
						x2,
						y2
					]
				)
		if not len(prediction):
			prediction.append([image_pth] + ["NEG", 1., 0, 0, 0, 0,])
		predictions.extend(prediction)
	return pd.DataFrame(
		predictions, columns=["Image_ID", "class", "confidence", "xmin", "ymin", "xmax", "ymax"]
	)

In [9]:
from tqdm import tqdm

In [ ]:
batch_size = 16
test_images = test["Image_ID"].unique()
results = [
	predicts(test_images[i : i + batch_size])
	for i in tqdm(
		range(0, len(test_images), batch_size), total=len(test_images) // batch_size + 1
	) if i < len(test_images)
]

In [11]:
predictions = pd.concat(results, ignore_index=True)

In [ ]:
predictions.sample(10)

In [ ]:
predictions["class"].value_counts(True)

In [14]:
predictions.to_csv(f"zindi_data/test/prediction_{FILE_NAME}.csv", index=False)

In [ ]:
f"zindi_data/test/prediction_{FILE_NAME}.csv"